# Form 5500 Querying using OpenAI and LangChain
## Step 1: Installing OpenAI and LangChain Packages

First we install the OpenAI and LangChain packages that will be used to import necessary modules.

In [59]:
!pip install langchain
!pip install openai

## Step 2: Importing Necessary Modules
Next we will import OpenAI, SQLDatabase, SQLDatabaseChain modules that will be used to work with a SQL Database.

In [60]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

## Step 3: Setting the OpenAI API key
Now we set our OpenAI API key to use the OpenAI API as an LLM for constructing SQL queries from an English sentence and then use the OpenAI completion model to convert the retrieved result from database into an English sentence and present it to the user.


In [61]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPEN_AI_API_KEY" #Please provide your OpenAI API key

## Step 4: Creating SQL Database Connection and OpenAI Language Mode
First we create an instance of the SQLDatabase class to establish a connection to an MSSQL database.
We can change the URI to match different specific database connection details.

Then we create an instance of the OpenAI class, representing the OpenAI language model. The temperature parameter sets the model's response generation randomness to zero since we are using OpenAI to construct SQL queries we dont want any creativity or randomness in this step, and the verbose parameter enables verbose output.

In [62]:
import pyodbc
username = "YOUR_MSSQL_USERNAME" #Enter MSSQL username
password = "YOUR_MSSQL_PASSWORD" #Enter MSSQL password
server = "YOUR_MSSQL_SERVER_NAME" #Enter MSSQL Server name
database = "DATABSE_NAME" #Enter database name

db = SQLDatabase.from_uri(f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server")
llm = OpenAI(temperature=0, verbose=True)
#include_tables=['sample'] This is optional, if you have multiple tables in the db, we can specify the required one.

## Step 5: Creating SQL Database Chain
Now we create an instance of the SQLDatabaseChain class, which combines the language model and the SQL database connection. The from_llm method initializes the chain with the provided language model (llm) and the SQL database connection (db). The verbose parameter enables verbose output.

In [63]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

## Step 6: Executing SQL Query
Now we pass a query in English to the run method to execute a SQL query on the database.

In [65]:
db_chain.run("How much is the total commission amount for all plans in Virginia in the year 2016?");

#Expected Output:
#SQLQuery:SELECT SUM("Commission") FROM form5500 WHERE "SPONS_MAIL_US_STATE" = 'VA' AND "FORM_PLAN_YEAR_BEGIN_DATE" BETWEEN '01-01-2016' AND '31-12-2016';
#SQLResult: [(101383,)]
#Answer:The total commission amount for all plans in Virginia in the year 2016 is 101383.



> Entering new SQLDatabaseChain chain...
total number of records
SQLQuery:SELECT COUNT(*) FROM [TodoList]
SQLResult: [(5,)]
Answer:There are 5 records in the TodoList table.
> Finished chain.


When we pass a query in form of an English sentence, LLM retrieves the schema of the database to contruct a SQL query rather than retrieving the whole database which would otherwise exceed the token limit of OpenAI and also would be a very expensive process for each query.

After contructing the SQL query, LangChain runs the SQL query on the provided Database and retrieves the result.

The result from the SQL query is then used with OpenAI Completion model to provide a structured answer using the Initial English query as context.

## Another Example

In [40]:
db_chain.run("Total number of organizations offering 401k plans?");



> Entering new SQLDatabaseChain chain...
Total number of organizations offering 401k plans?
SQLQuery:SELECT COUNT(*) FROM form5500 WHERE TYPE_PENSION_BNFT_CODE LIKE '%2E2F2G2J2K2R3D3H%';
SQLResult: [(119,)]
Answer:119 organizations are offering 401k plans.
> Finished chain.


In [42]:
db_chain.run("Organizations offering dental plans in Washington in the year 2017");



> Entering new SQLDatabaseChain chain...
Organizations offering dental plans in Washington in the year 2017
SQLQuery:SELECT "PLAN_NAME", "SPONSOR_NAME", "SPONS_MAIL_US_STATE" FROM form5500 WHERE "TYPE_WELFARE_BNFT_CODE" LIKE '%D%' AND "SPONS_MAIL_US_STATE" = 'WA' AND "FORM_PLAN_YEAR_BEGIN_DATE" BETWEEN '01-01-2017' AND '31-12-2017' LIMIT 5;
SQLResult: [('AFFIRMA, LLC WELFARE BENEFITS PLAN', 'AFFIRMA, LLC', 'WA'), ('ALTEK, INC. HEALTH & WELFARE PLANS', 'ALTEK, INC.', 'WA'), ('ARCTIC STORM, INC WELFARE BENEFIT PLAN', 'ARCTIC STORM, INC.', 'WA'), ('ASKO PROCESSING, INC. EMPLOYEE BENEFIT PLAN', 'ASKO PROCESSING, INC.', 'WA'), ('ASTRIA HEALTH', 'ASTRIA HEALTH', 'WA')]
Answer:There are 5 organizations offering dental plans in Washington in the year 2017: AFFIRMA, LLC WELFARE BENEFITS PLAN, ALTEK, INC. HEALTH & WELFARE PLANS, ARCTIC STORM, INC WELFARE BENEFIT PLAN, ASKO PROCESSING, INC. EMPLOYEE BENEFIT PLAN, and ASTRIA HEALTH.
> Finished chain.
